# Manipulação de valores numericos no Apache Spark

O Apache Spark fornece uma variedade de funções matemáticas para manipular valores inteiros e de ponto flutuante em DataFrames. Essas funções podem ser usadas para realizar operações aritméticas, agregações e transformações de dados.


| Função               | Descrição                                                        | Exemplo de Caso de Uso                |
|----------------------|------------------------------------------------------------------|---------------------------------------|
| `abs(col)`           | Valor absoluto de uma coluna                                    | Calcular distância, diferença de valores. |
| `ceil(col)`          | Teto (arredondamento para cima) de uma coluna                    | Arredondar para a próxima unidade superior. |
| `floor(col)`         | Piso (arredondamento para baixo) de uma coluna                    | Arredondar para a próxima unidade inferior. |
| `round(col, scale)`  | Arredondamento de uma coluna com precisão específica             | Arredondar para um número específico de casas decimais. |
| `greatest(col1, col2, ...)` | O maior valor entre as colunas                     | Encontrar o maior valor em várias colunas. |
| `least(col1, col2, ...)`   | O menor valor entre as colunas                      | Encontrar o menor valor em várias colunas. |
| `sqrt(col)`          | Raiz quadrada de uma coluna                                      | Calcular magnitude, desvio padrão.       |
| `cbrt(col)`          | Raiz cúbica de uma coluna                                         | Calcular volume de um cubo.              |
| `exp(col)`           | Exponencial de uma coluna                                         | Modelagem de crescimento exponencial.    |
| `log(col, base)`     | Logaritmo de uma coluna em uma base específica                   | Análise de dados em escala logarítmica.  |
| `log10(col)`         | Logaritmo base 10 de uma coluna                                   | Análise de dados em escala logarítmica.  |
| `pow(col, exponent)` | Eleva uma coluna à potência especificada                        | Cálculos complexos de potência.          |


In [1]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("Exemplo").getOrCreate()

In [2]:
df = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)


In [3]:
df.show()

+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+
|icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|partida_prevista|    partida_real|chegada_prevista|    chegada_real|situacao_voo|codigo_justificativa|
+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+
|               TAM|      9458|        0|                I|                 SBGR|                  SCEL|01/01/2020 06:30|01/01/2020 06:30|01/01/2020 10:40|01/01/2020 10:40|   REALIZADO|                null|
|               TAM|      9201|        0|                N|                 SBSG|                  SBGL|01/01/2020 02:00|01/01/2020 02:00|01/01/2020 05:10|01/01/2020 05:10|

# Primeiramente vamos criar valores representando o valor do preço dos voos e o tanto de combustivel utilizado em cada rota

In [4]:
from pyspark.sql.functions import rand 

valorMaximo1 = 7499.87
valorMinimo1 = 433.47

valorMaximo2 = 20000.00
valorMinimo2 = 11499.67

df_com_coluna_variavel = (
    df
    .withColumn("preco_passagem", (rand() * (valorMaximo1 - valorMinimo1) + valorMinimo1).cast("float"))
    .withColumn("preco_combustivel", (rand() * (valorMaximo2 - valorMinimo2) + valorMinimo2).cast("double"))
)

In [5]:
df_com_coluna_variavel.show()

+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+--------------+------------------+
|icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|partida_prevista|    partida_real|chegada_prevista|    chegada_real|situacao_voo|codigo_justificativa|preco_passagem| preco_combustivel|
+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+--------------+------------------+
|               TAM|      9458|        0|                I|                 SBGR|                  SCEL|01/01/2020 06:30|01/01/2020 06:30|01/01/2020 10:40|01/01/2020 10:40|   REALIZADO|                null|      485.1993|16359.611879427888|
|               TAM|      9201|     

In [19]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

(
    df_com_coluna_variavel
    .withColumn("ceil_preço", ceil(df_com_coluna_variavel["preco_passagem"]))
    .withColumn("floor_preço", floor(df_com_coluna_variavel["preco_passagem"]))
    .withColumn("round_preço", round(df_com_coluna_variavel["preco_passagem"],1))
    .withColumn("somatori_squares_preço", cbrt(df_com_coluna_variavel["preco_passagem"])+sqrt(df_com_coluna_variavel["preco_passagem"]))
    .withColumn("pow_preço", pow(df_com_coluna_variavel["preco_passagem"],3))
    .withColumn("valor_unitario", lit(1))
    .select("valor_unitario","preco_passagem","ceil_preço","floor_preço","round_preço","pow_preço","somatori_squares_preço",)
    .show()
)

+--------------+--------------+----------+-----------+-----------+--------------------+----------------------+
|valor_unitario|preco_passagem|ceil_preço|floor_preço|round_preço|           pow_preço|somatori_squares_preço|
+--------------+--------------+----------+-----------+-----------+--------------------+----------------------+
|             1|      485.1993|       486|        485|      485.2| 1.142248311050879E8|    29.885144300593787|
|             1|     3578.3142|      3579|       3578|     3578.3|4.581792520120428E10|     75.11436462798598|
|             1|      814.9517|       815|        814|      815.0| 5.412471767238849E8|     37.88801345531599|
|             1|      3125.884|      3126|       3125|     3125.9|3.054348486263903...|     70.53107310147381|
|             1|     2596.9507|      2597|       2596|     2597.0|1.751423236190595...|     64.70559625736264|
|             1|     3414.3005|      3415|       3414|     3414.3|3.980203150617813E10|     73.49001484443045|
|